In [26]:
!pip install openai

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 44 kB 807 kB/s  eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 146 kB 79.1 MB/s eta 0:00:01
     |████████████████████████████████| 242 kB 99.5 MB/s eta 0:00:01
     |████████████████████████████████| 12.2 MB 91.8 MB/s eta 0:00:01
  Created wheel for openai: filename=openai-0.25.0-py3-none-any.whl size=55859 sha256=a1f8e724d89e5366da5dab67a71b0d6d4168aaee51b47f14b2ba09e1238fcb63
  Stored in directory: /home/jnw301/.cache/pip/wheels/4b/92/33/6f57c7aae0b16875267999a50570e81f15eecec577ebe05a2e
Successfully built openai
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test yo

In [36]:
import os
import sys
sys.path.append("../")

import pickle
import json
import glob
from tqdm.auto import trange, tqdm
import openai
from features import merge_entries, prepare_entry
import nltk
from utills import chunker, get_num_chunks
import numpy as np
from nltk.tokenize import sent_tokenize

In [7]:
PREPROCESSED_DATA_PATH = '/scratch/jnw301/authorship_verification/temp_data/pan/'

In [20]:
def merge_and_get_text(entries):
    return ' '.join([e['preprocessed'] for e in entries])

In [72]:

path = PREPROCESSED_DATA_PATH + 'preprocessed_test.jsonl'
max_length = 1024
sent_chunk_sz = 50
max_fanfics = 10
prompts = []
i = 0
with open(path, 'r') as f:
    for l in tqdm(f, total=max_fanfics//2):
        d = json.loads(l)
        chunk = list(chunker(sent_tokenize(merge_and_get_text(d['pair'][0])), sent_chunk_sz))[0]
        text = ' '.join(chunk)
        prompt = f"Complete the rest of this fanfiction:\n{text}\n\n"
        prompts.append(prompt)
        
        chunk = list(chunker(sent_tokenize(merge_and_get_text(d['pair'][1])), sent_chunk_sz))[0]
        text = ' '.join(chunk)
        prompt = f"Complete the rest of this fanfiction:\n{text}\n\n"
        i += 2
        if i > max_fanfics:
            break

  0%|          | 0/5 [00:00<?, ?it/s]

In [68]:
openai.api_key = ""
response = openai.Completion.create(
  model="text-davinci-002",
  prompt=prompts[1],
  temperature=0.1,
  max_tokens=2048,
  top_p=1,
  frequency_penalty=0.1,
  presence_penalty=0
)

In [73]:
print(prompts[3])

Complete the rest of this fanfiction:
I suppose one could argue that what happened to me was fair. One might even argue that it wasn"t enough. Wasn"t I the one responsible for the death of countless innocents, certainly numbering into the tens of thousands? Yet here I am, walking away from the war with my life intact, though I cannot say the same for my face. I had personally viewed the bodies of viciously beaten and scalped Nazis, courtesy of the very group I had somehow depended on to vindicate me. These slaughtered men were merely foot soldiers, men simply carrying out wartime duties. Bakers, tailors, shopkeepers, blacksmiths. I, on the other hand, was for all intents and purposes, a mercenary, one who had spent years earning my infamous nickname. I was personally responsible for the deaths of innocent men, women and children who had never wronged me in any way. And yet, I"m alive, by my own volition. Obviously the end was nigh: millions upon millions of Russians were practically at

In [69]:
print(response.to_dict()['choices'][0]['text'])



"So, you spank Amy when she does something bad?" Leonard asked, still trying to wrap his head around the concept. "Yes, I spank her," Sheldon confirmed. "It's the only thing that gets her to behave." "But, Sheldon, don't you think that's a little... abusive?" Leonard asked, his brow furrowed in concern. "No, I don't think it's abusive," Sheldon replied. "I think it's the only way to get her to behave." "But, Sheldon, she's a grown woman," Leonard pointed out. "She doesn't need you to spank her like a child." "I beg to differ," Sheldon said. "Amy is very childish and immature. She needs someone to help her grow up and learn how to behave." "But, Sheldon, you can't just go around spanking her whenever she does something you don't like," Leonard said. "It's not fair to her." "I'm not doing it because I don't like it," Sheldon said. "I'm doing it because it's the only thing that gets her to behave." "But, Sheldon, you can't just hit her whenever she does something wrong," Leonard said. "

In [70]:
len(response.to_dict()['choices'][0]['text'].split(' '))

227

In [41]:
response.keys()

dict_keys(['id', 'object', 'created', 'model', 'choices', 'usage'])

In [34]:
for p in prompts:
    print(p)

Generate the rest of this Dr. Horrible's Sing-Along Blog fanfiction:
"Goodbye Dr. Horrible" I frowned, trying to hold back tears. I took Sid"s arm and left with no intention of ever returning. When we reached outside though, I could feel my spirit breaking. Sid put her hand on my shoulder sympathetically. "Hey, it"s okay Penny. Maybe it"s for the best." She tried to be optimistic. "Maybe you will find somebody else, so what if he is probably the best guy for you ever. Sorry Penny but you two were probably the cutest thing I"d ever seen for about two seconds there. But at the end of the day, he"s a bad guy and you aren"t so you know...oh well." I froze, realizing that I really had to listen to Sid for once. I learned that I had to join them if you couldn"t beat them...clearly Billy wasn"t going to stop being evil...it"s who he is I guess... I spun around and hugged my cousin. She didn"t realize I was saying goodbye until I had walked into the building. "Go get him Penny...I think!" Sid 

In [27]:
list(chunker(sent_tokenize(merge_and_get_text(d['pair'][0])), sent_chunk_sz))

[['"Goodbye Dr. Horrible" I frowned, trying to hold back tears.',
  'I took Sid"s arm and left with no intention of ever returning.',
  'When we reached outside though, I could feel my spirit breaking.',
  'Sid put her hand on my shoulder sympathetically.',
  '"Hey, it"s okay Penny.',
  'Maybe it"s for the best."',
  'She tried to be optimistic.',
  '"Maybe you will find somebody else, so what if he is probably the best guy for you ever.',
  'Sorry Penny but you two were probably the cutest thing I"d ever seen for about two seconds there.',
  'But at the end of the day, he"s a bad guy and you aren"t so you know...oh well."',
  'I froze, realizing that I really had to listen to Sid for once.',
  'I learned that I had to join them if you couldn"t beat them...clearly Billy wasn"t going to stop being evil...it"s who he is I guess...',
  'I spun around and hugged my cousin.',
  'She didn"t realize I was saying goodbye until I had walked into the building.',
  '"Go get him Penny...I think!"'

In [24]:
prompt = """"""
print(dataset[0])

"Goodbye Dr. Horrible" I frowned, trying to hold back tears. I took Sid"s arm and left with no intention of ever returning. When we reached outside though, I could feel my spirit breaking. Sid put her hand on my shoulder sympathetically. "Hey, it"s okay Penny. Maybe it"s for the best." She tried to be optimistic. "Maybe you will find somebody else, so what if he is probably the best guy for you ever. Sorry Penny but you two were probably the cutest thing I"d ever seen for about two seconds there. But at the end of the day, he"s a bad guy and you aren"t so you know...oh well." I froze, realizing that I really had to listen to Sid for once. I learned that I had to join them if you couldn"t beat them...clearly Billy wasn"t going to stop being evil...it"s who he is I guess... I spun around and hugged my cousin. She didn"t realize I was saying goodbye until I had walked into the building. "Go get him Penny...I think!" Sid shouted after me before leaving, finally intent on getting her house 

In [13]:
print(dataset[0])

"Goodbye Dr. Horrible" I frowned, trying to hold back tears. I took Sid"s arm and left with no intention of ever returning. When we reached outside though, I could feel my spirit breaking. Sid put her hand on my shoulder sympathetically. "Hey, it"s okay Penny. Maybe it"s for the best." She tried to be optimistic. "Maybe you will find somebody else, so what if he is probably the best guy for you ever. Sorry Penny but you two were probably the cutest thing I"d ever seen for about two seconds there. But at the end of the day, he"s a
bad guy and you aren"t so you know...oh well." I froze, realizing that I really had to listen to Sid for once. I learned that I had to join them if you couldn"t beat them...clearly Billy wasn"t going to stop being evil...it"s who he is I guess... I spun around and hugged my cousin. She didn"t realize I was saying goodbye until I had walked into the building. "Go get him Penny...I think!" Sid shouted after me before leaving, finally intent on getting her house 